In [1]:
%load_ext jupyter_black

In [5]:
import getpass
import os
import instructor
from pydantic import BaseModel
from openai import OpenAI
from typing import Iterable, Literal

In [3]:
_API_KEY = getpass.getpass("Your API key: \n")

Your API key: 
 ········


In [16]:
open_ai = OpenAI(api_key=_API_KEY)
client = instructor.from_openai(open_ai, mode=instructor.Mode.TOOLS_STRICT)

In [35]:
psu_info = """
As Pennsylvania's only land-grant university, Penn State has a broad mission of teaching, research, and public service. 
But that mission was not so grandly conceived in 1855, when the Commonwealth chartered it as a college of agricultural
science to apply scientific principles to farming.

Centre County became the site of the new college in response to a gift of 200 acres from gentleman farmer 
and ironmaster James Irvin of Bellefonte. Founding President Evan Pugh drew on the scientific education he
had received in Europe to plan a curriculum that combined theoretical studies with practical applications.

Pugh and similar visionaries in other states championed Congressional passage of the Morrill Land-Grant Act in 1862. 
The act enabled states to sell federal land, invest the proceeds, and use the income to support colleges 
"where the leading object shall be, without excluding scientific and classical studies ... to teach agriculture 
and the mechanic arts [engineering] ... in order to promote the liberal and practical education of the industrial 
classes in all the pursuits and professions of life." The state legislature designated Penn State the land-grant institution of Pennsylvania.

But not until the 1880s, under the leadership of President George W. Atherton, did the college expand 
its curriculum to match the Land-Grant Act's broad mandate. From that time onward, curriculums in engineering, 
the sciences, the liberal arts, and more began to flourish. In the early 1900s, Penn State introduced 
cooperative extension and additional outreach programming, extending the reach of its academic mission.

An even greater segment of the Commonwealth's population had opportunities for engagement in the 1930s
when Penn State established a series of undergraduate branch campuses, primarily to meet the needs of
students who were location-bound during the Great Depression. Those campuses were predecessors of today's
system of twenty-four Penn State campuses located throughout the Commonwealth.

Penn State began offering systematic advanced-degree work in 1922 with the formation of the Graduate School 
(now the J. Jeffrey and Ann Marie Fox Graduate School). Graduate education and research evolved hand in hand. 
By 1950 the University had won international distinction for investigations in dairy science, building insulation,
diesel engines, and acoustics, and other specialized fields.

A college of medicine and teaching hospital were established in 1967 with a $50 million gift from the
charitable trusts of renowned chocolate magnate Milton S. Hershey. In 1989, the Pennsylvania College of Technology in 
Williamsport became an affiliate of the University. In 2000, Penn State and the Dickinson School of Law merged, 
meaning the University now operates two law schools—Dickinson Law (in Carlisle, Pennsylvania) and Penn State Law (on the University Park campus). 
Penn State's online World Campus graduated its first students in 2000. 
"""

In [13]:
psu_info_chunked = """
<quote>
<title>section-01</title>
As Pennsylvania's only land-grant university, Penn State has a broad mission of teaching, research, and public service. 
But that mission was not so grandly conceived in 1855, when the Commonwealth chartered it as a college of agricultural
science to apply scientific principles to farming.
</quote>

<quote>
<title>section-02</title>
Centre County became the site of the new college in response to a gift of 200 acres from gentleman farmer 
and ironmaster James Irvin of Bellefonte. Founding President Evan Pugh drew on the scientific education he
had received in Europe to plan a curriculum that combined theoretical studies with practical applications.
</quote>

<quote>
<title>section-03</title>
Pugh and similar visionaries in other states championed Congressional passage of the Morrill Land-Grant Act in 1862. 
The act enabled states to sell federal land, invest the proceeds, and use the income to support colleges 
"where the leading object shall be, without excluding scientific and classical studies ... to teach agriculture 
and the mechanic arts [engineering] ... in order to promote the liberal and practical education of the industrial 
classes in all the pursuits and professions of life." The state legislature designated Penn State the land-grant institution of Pennsylvania.
</quote>

<quote>
<title>section-04</title>
But not until the 1880s, under the leadership of President George W. Atherton, did the college expand 
its curriculum to match the Land-Grant Act's broad mandate. From that time onward, curriculums in engineering, 
the sciences, the liberal arts, and more began to flourish. In the early 1900s, Penn State introduced 
cooperative extension and additional outreach programming, extending the reach of its academic mission.
</quote>

<quote>
<title>section-05</title>
An even greater segment of the Commonwealth's population had opportunities for engagement in the 1930s
when Penn State established a series of undergraduate branch campuses, primarily to meet the needs of
students who were location-bound during the Great Depression. Those campuses were predecessors of today's
system of twenty-four Penn State campuses located throughout the Commonwealth.
</quote>

<quote>
<title>section-06</title>
Penn State began offering systematic advanced-degree work in 1922 with the formation of the Graduate School 
(now the J. Jeffrey and Ann Marie Fox Graduate School). Graduate education and research evolved hand in hand. 
By 1950 the University had won international distinction for investigations in dairy science, building insulation,
diesel engines, and acoustics, and other specialized fields.
</quote>

<quote>
<title>section-07</title>
A college of medicine and teaching hospital were established in 1967 with a $50 million gift from the
charitable trusts of renowned chocolate magnate Milton S. Hershey. In 1989, the Pennsylvania College of Technology in 
Williamsport became an affiliate of the University. In 2000, Penn State and the Dickinson School of Law merged, 
meaning the University now operates two law schools—Dickinson Law (in Carlisle, Pennsylvania) and Penn State Law (on the University Park campus). 
Penn State's online World Campus graduated its first students in 2000.
</quote>
"""

In [12]:
# from https://mattyyeung.github.io/deterministic-quoting
system_prompt = """You are an expert Q&A system with excellent knowledge.
It is critical that you always accurately quote the reference information you use to answer the query."""

In [14]:
# from https://mattyyeung.github.io/deterministic-quoting
user_instruction = f"""Always answer the query using the provided reference information, and not prior knowledge.
The reference information is provided below as a list of "sections". Each section is encapsulated by a <quote> tag.
Each <quote> contains a <title> tag at the very beginning. Use this title when quoting.

Some rules to follow:
1. Always use the appropriate <quote>s to answer the question.
2. Include a plain English summary in answer to the query, based on the text in the relevant <quote>s. There is no need to simply repeat the quoted text, however.
3. Quote the whole section so the user can see where the information has come from. Always include the <quote> tags, including the title, when quoting.
4. Always start quotes with "<quote>" and end quotes with "</quote>"
5. Quote multiple sections if relevant, but always quote whole sections using the correct format.
6. Use the term "Reference information" instead of "context information"
7. Always quote whole sections verbatim, not a subset.

"""

In [10]:
class Response(BaseModel):
    message: str

In [17]:
def get_response(developer_instruction, user_instruction, chunks, query):
    return client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=Response,
        messages=[
            {
                "role": "developer",
                "content": f"{developer_instruction}",
            },
            {
                "role": "user",
                "content": f"{user_instruction}",
            },
            {
                "role": "user",
                "content": f"{chunks}",
            },
            {
                "role": "user",
                "content": f"Query: {query}",
            },
        ],
    )

In [24]:
q = "What's the name of the graduate school?"
r = get_response(
    system_prompt,
    user_instruction,
    psu_info_chunked,
    q,
)

In [25]:
print(r.message)

The name of the graduate school is the J. Jeffrey and Ann Marie Fox Graduate School. This information can be referenced from the following section:

<quote>
<title>section-06</title>
Penn State began offering systematic advanced-degree work in 1922 with the formation of the Graduate School 
(now the J. Jeffrey and Ann Marie Fox Graduate School). Graduate education and research evolved hand in hand. 
By 1950 the University had won international distinction for investigations in dairy science, building insulation,
diesel engines, and acoustics, and other specialized fields.
</quote>


In [26]:
q = "What's the location of Pennsylvania College of Technology?"
r = get_response(
    system_prompt,
    user_instruction,
    psu_info_chunked,
    q,
)

In [27]:
print(r.message)

The Pennsylvania College of Technology is located in Williamsport, Pennsylvania. This information is derived from the following section of the reference information:

<quote>
<title>section-07</title>
A college of medicine and teaching hospital were established in 1967 with a $50 million gift from the charitable trusts of renowned chocolate magnate Milton S. Hershey. In 1989, the Pennsylvania College of Technology in Williamsport became an affiliate of the University. In 2000, Penn State and the Dickinson School of Law merged, meaning the University now operates two law schools—Dickinson Law (in Carlisle, Pennsylvania) and Penn State Law (on the University Park campus). Penn State's online World Campus graduated its first students in 2000.
</quote>


In [28]:
q = "What's the location of College of Information Sciences and Technology?"
r = get_response(
    system_prompt,
    user_instruction,
    psu_info_chunked,
    q,
)

In [29]:
print(r.message)

The reference information provided does not specifically mention the location of the College of Information Sciences and Technology at Penn State. Therefore, I cannot provide an answer based on the given quotes.


# ToDo: When does it fail?

* Doesn't provide answer when the information is contained there
* Hallucinate

# ToDo: Retrieve Palliative Care Guideline

We will use the [Care Management by Oncology Nurses (CONNECT) intervention](https://pmc.ncbi.nlm.nih.gov/articles/PMC4347888/) guideline.
It provides recommendations for managing differnt symptoms for individuals with advanced cancer (in  palliative care). For this activity,
we will focus on fatigue guidelines (in Canvas -> Files -> Activity -> Activity-06 -> fatigue.docx).

You will write a function that will take a fatigue assessment (0–10 scale) and return a guideline. E.g.:

```
def get_fatigue_guideline(fatigue: int) -> str:
    return f"{fatigue}"
```

* It is impretative the guideline is correct
    +  It must be from the provided guidelines (i.e., no external resource)
    +  It must correspond to a given fatigue level
* The guideline must be verbatim (i.e., no added text)
* For a given fatigue level, the guideline must be random (we can't just return the first guideline)

## Strategy
* How would you chunk the given document?
* How would you strip any chunking information or reference (note that you will have to return verbatim guideline)
* How would you ensure randomized selection?
* Can CoT/few-shot prompting be useful here?



# ToDo: Generate intervention messages

Now that you can correctly extract guideline for a given fatigue level, let's generate intervention messages
for a patient based on the extracted guideline. E.g.:

```
def create_intervention(fatigue: int) -> str:
    guideline = get_fatigue_guideline(fatigue)
    ...
```

The generated intervention must be:

* accurate
* less than 160 characters
* empathetic, providing emotional and spiritual support
* fifth grade readability level ([why?](https://www.thenationalliteracyinstitute.com/post/literacy-statistics-2024-2025-where-we-are-now#:~:text=On%20average%2C%2079%25%20of%20U.S.,US%20ranks%2036th%20in%20literacy.))
